In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.datasets import cifar10

import tensorflow as tf
import numpy as np

In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 32, 32, 3))
x_test = np.reshape(x_test, (len(x_test), 32, 32, 3))

print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [5]:
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

In [6]:
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [7]:
input_img = Input(shape = (32, 32, 3))

In [8]:
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

In [9]:
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(3,(3,3), activation='sigmoid', padding='same')(x)

In [10]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mae')

In [ ]:
autoencoder.fit(x_train_noisy, x_train, 
                epochs=100, 
                batch_size=128, 
                shuffle=True, 
                validation_data=(x_test_noisy,x_test))

Epoch 1/100
391/391 [==============================] - 6s 16ms/step - loss: 0.1180 - val_loss: 0.0998
Epoch 2/100
391/391 [==============================] - 6s 15ms/step - loss: 0.0966 - val_loss: 0.0944
Epoch 3/100
391/391 [==============================] - 6s 16ms/step - loss: 0.0929 - val_loss: 0.0914
Epoch 4/100
391/391 [==============================] - 6s 15ms/step - loss: 0.0909 - val_loss: 0.0900
Epoch 5/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0891 - val_loss: 0.0893
Epoch 6/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0880 - val_loss: 0.0872
Epoch 7/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0870 - val_loss: 0.0864
Epoch 8/100
391/391 [==============================] - 9s 24ms/step - loss: 0.0863 - val_loss: 0.0859
Epoch 9/100
391/391 [==============================] - 6s 15ms/step - loss: 0.0857 - val_loss: 0.0850
Epoch 10/100
391/391 [==============================] - 6s 14ms/step - loss: 0.085

391/391 [==============================] - 6s 14ms/step - loss: 0.0778 - val_loss: 0.0786
Epoch 81/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0778 - val_loss: 0.0783
Epoch 82/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0777 - val_loss: 0.0780
Epoch 83/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0778 - val_loss: 0.0781
Epoch 84/100
391/391 [==============================] - 6s 15ms/step - loss: 0.0778 - val_loss: 0.0780
Epoch 85/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0776 - val_loss: 0.0780
Epoch 86/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0777 - val_loss: 0.0779
Epoch 87/100
391/391 [==============================] - 6s 16ms/step - loss: 0.0776 - val_loss: 0.0779
Epoch 88/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0776 - val_loss: 0.0783
Epoch 89/100
391/391 [==============================] - 6s 14ms/step - loss: 0.0776 - 

In [ ]:
decoded_imgs = autoencoder.predict(x_test_noisy)
print(decoded_imgs.shape)

In [ ]:
import matplotlib.pyplot as plt

n = 5
plt.figure(figsize=(40, 16))
for i in range(n):
  #display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test_noisy[i].reshape(32, 32, 3))
  #plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  #display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i].reshape(32,32, 3))
  #plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)